In [1]:
# ok bd de mensagens
    # ok remover mensagens ja enviadas
# ok pegar mensagem random
# TODO enviar tgrm
# TODO agendar no serv para enviar td dia
# TODO add mais frases pelo menos 360

# TODO segundo script - rodar de hora em hora com lembretes - relacionar com planilha

# ok anotar como mensagem enviada - salva db


import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
import sys
# pip install python-dotenv
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()



agora = datetime.now().isoformat(sep=' ', timespec='seconds')



In [2]:
# DB
conn = sqlite3.connect('db_msgs.db')  # Arquivo .db será criado no diretório atual
cursor = conn.cursor()

# Criação da tabela com id, numero e timestamp

# 🚨 ADMIN
# limpar tabela 🚨🚨🚨🚨
DROP = False

if DROP == True:

    cursor.execute("""
    DROP TABLE eventos 
    """)

cursor.execute("""
CREATE TABLE IF NOT EXISTS eventos (

    id INTEGER PRIMARY KEY AUTOINCREMENT,
    numero_frase INTEGER NOT NULL,
    timestamp TEXT NOT NULL
)
""")

conn.commit()



In [3]:
# sortear frase a ser enviada


# TODO filtrar frases ja enviadas --------

# Conectar ao banco
# conn = sqlite3.connect('db_msgs.db')

# Ler a tabela como DataFrame
df_msgs_enviadas = pd.read_sql_query("SELECT * FROM eventos", conn)

# # Exibir o DataFrame
# display(df_msgs_enviadas)
# # Fechar conexão
# conn.close()

# ----------------------------------------

# DF de mensagens a sortear
df_msgs = pd.read_csv("db_msgs2.csv", sep=";")

    # filtrar mensagens já enviadas
df_msgs_filtro = df_msgs[~df_msgs['ID'].isin(df_msgs_enviadas['numero_frase'])]


if len(df_msgs_filtro) == 0:
    print("🚨🚨🚨 Acabaram as frases!! 🚨🚨🚨")
    #TODO mandar essa frase no telegram
    sys.exit()

else:
    numero_frase = int(df_msgs_filtro['ID'].sample(1).iloc[0])
    print(f"Frase Sorteada: {numero_frase}")

Frase Sorteada: 19


In [4]:
len(df_msgs_filtro)

48

In [5]:
# output construção da frase 


mensagem = df_msgs[df_msgs['ID'] == numero_frase]['Mensagem'].iloc[0]
autor = df_msgs[df_msgs['ID'] == numero_frase]['Autor/Fonte'].iloc[0]

mensagem_ouput = f"{mensagem} - {autor}"



print(mensagem_ouput)


# TODO envio telegrm



# TODO adicionar as frases enviadas

cursor.execute("INSERT INTO eventos (numero_frase, timestamp) VALUES (?, ?)", (numero_frase, agora))
conn.commit()
# conn.close()
print(f"Registro inserido: frase #{numero_frase} em {agora}")




A felicidade não é algo pronto. Ela vem de suas próprias ações. - Dalai Lama
Registro inserido: frase #19 em 2025-09-16 22:29:31


In [6]:
# Visualizar no db


# Conectar ao banco
conn = sqlite3.connect('db_msgs.db')

# Ler a tabela como DataFrame
df_msgs_enviadas = pd.read_sql_query("SELECT * FROM eventos", conn)

# Exibir o DataFrame
display(df_msgs_enviadas)
# Fechar conexão
conn.close()

,id,numero_frase,timestamp
0,1,48,2025-09-16 21:38:08
1,2,46,2025-09-16 21:38:21
2,3,2,2025-09-16 21:38:25
3,4,60,2025-09-16 21:38:34
4,5,32,2025-09-16 22:02:08
5,6,58,2025-09-16 22:03:26
6,7,20,2025-09-16 22:08:55
7,8,51,2025-09-16 22:13:39
8,9,40,2025-09-16 22:19:30
9,10,53,2025-09-16 22:20:00


In [7]:
numero_frase

19

In [8]:
# TODO botar no .env

TOKEN = os.getenv("TOKEN")
CHAT_ID = os.getenv("CHAT_ID")

In [9]:
mensagem_ouput

'A felicidade não é algo pronto. Ela vem de suas próprias ações. - Dalai Lama'

In [10]:
# enviar a mensagem para o Telegram

from telegram import Bot

bot = Bot(token=TOKEN)
await bot.send_message(chat_id=CHAT_ID, text=mensagem_ouput)






Message(channel_chat_created=False, chat=Chat(first_name='A', id=7988991522, last_name='R', type=<ChatType.PRIVATE>), date=datetime.datetime(2025, 9, 17, 1, 29, 2, tzinfo=datetime.timezone.utc), delete_chat_photo=False, from_user=User(first_name='R. Assistant', id=8117396962, is_bot=True, username='AR20_AssistantBot'), group_chat_created=False, message_id=11, supergroup_chat_created=False, text='A felicidade não é algo pronto. Ela vem de suas próprias ações. - Dalai Lama')

In [11]:
# async def enviar(texto):
#     async with Bot(TOKEN) as bot:
#         await bot.send_message(chat_id=CHAT_ID, text=mensagem_ouput)